In [1]:
from datasets.wider_part_dataset import build_wider_dataloader
from datasets.text_test_datasets import build_text_test_loader
from datasets.image_test_datasets import build_image_test_loader
from models.encoder import Model, MLP
from evaluators.global_evaluator import GlobalEvaluator
from evaluators.np_evaluator import NPEvaluator
from loss.loss import crossmodal_triplet_loss, cos_distance, triplet_cos_loss
from loggers.logger import Logger
from manager import build_graph_optimizer
from tqdm import tqdm_notebook as tqdm
from sklearn.neighbors import DistanceMetric

from attentions.rga_attention import RGA_attend_one_to_many_batch, RGA_attend_one_to_many
import os

import torch.nn as nn
import torch.optim as optim

from configs.args import load_arg_parser

In [3]:
parser = load_arg_parser()
cfg = parser.parse_args("")
cfg.data_root = "/data/aiyucui2/wider"
root = cfg.data_root

# data path
cfg.anno_path = os.path.join(root, cfg.anno_path)
cfg.img_dir = os.path.join(root, cfg.img_dir)
cfg.val_anno_path = os.path.join(root, cfg.val_anno_path)
cfg.val_img_dir = os.path.join(root, cfg.val_img_dir)
cfg.gt_file_fn = os.path.join(root, cfg.gt_file_fn)

# meta data path
cfg.cheap_candidate_fn = os.path.join(root, cfg.cheap_candidate_fn)
cfg.vocab_path = os.path.join(root, cfg.vocab_path)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# sys path
cfg.model_path = os.path.join(root, cfg.model_path)
cfg.output_path = os.path.join(root, cfg.output_path)
ckpt_root = "/shared/rsaas/aiyucui2/wider_person/checkpoints/reID/baseline"
load_exp_name = "dist_fn_cosine_imgbb_resnet50_capbb_bigru_embed_size_1024_batch_96_lr_0.0001_captype_sent_img_meltlayer_2_cos_margin_0.2_np_False"
cfg.load_ckpt_fn = os.path.join(ckpt_root, load_exp_name, "stage_1_id_last.pt")
cfg.debug = False
cfg.embed_size = 1024
cfg.batch_size = 96
cfg.img_backbone_opt = "resnet50"
cfg.num_gpus = 1
cfg.cap_backbone_opt = "bigru"
cfg.dim = (384,128)
cfg.dist_fn_opt = "cosine"
cfg.np = False
cfg.img_num_cut = 6
cfg.img_num_cut = 1 if not cfg.np else cfg.img_num_cut
cfg.np_token_length = 5
cfg.sent_token_length =30

cfg.cap_embed_type='sent'
# exp_name
cfg.exp_name = 'debug'
cfg.model_path = os.path.join("/shared/rsaas/aiyucui2/wider_person", cfg.model_path, cfg.exp_name)
cfg.output_path = os.path.join("/shared/rsaas/aiyucui2/wider_person", cfg.output_path, cfg.exp_name)

if not os.path.exists(cfg.model_path):
    os.mkdir(cfg.model_path)
if not os.path.exists(cfg.output_path):
    os.mkdir(cfg.output_path)
# logger
logger = Logger("test_sent30.txt") #os.path.join(cfg.output_path, cfg.exp_name+".txt"))
print(cfg.exp_name)

FileNotFoundError: [Errno 2] No such file or directory: '/data/aiyucui2/wider/checkpoints/reID/debug'

In [ ]:
# train loader
train_loader = build_wider_dataloader(cfg)
# test loader (loading image and text separately)
test_text_loader = build_text_test_loader(cfg) 
test_image_loader = build_image_test_loader(cfg) 

# Evaluator
Evaluator = NPEvaluator if cfg.np else GlobalEvaluator

evaluator = Evaluator(img_loader=test_image_loader, 
                          cap_loader=test_text_loader, 
                          gt_file_path=cfg.gt_file_fn,
                          embed_size=cfg.embed_size,
                          logger=logger,
                          dist_fn_opt="euclidean")
cos_evaluator = Evaluator(img_loader=test_image_loader, 
                          cap_loader=test_text_loader, 
                          gt_file_path=cfg.gt_file_fn,
                          embed_size=cfg.embed_size,
                          logger=logger,
                          dist_fn_opt="cosine")

In [ ]:
#ckpt_path = "/shared/rsaas/aiyucui2/wider_person/checkpoints/reID/resnet18_bigru_512/resnet18_bigru_512_id_initalized.pt"
#ckpt = torch.load(ckpt_path)
cfg.num_gpus

In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim 
import os

import torch.nn.functional as F
from models.encoder import Model, MLP
from loss.loss import triplet_cos_loss, crossmodal_triplet_loss, triplet_cos_loss_inner

from attentions.rga_attention import RGA_attend_one_to_many_batch, RGA_attend_one_to_many
from models.texts.gru_backbone import BiGRUBackbone as CaptionBackbone
  
def triplet_cos_loss(imgs, caps, pids):
    loss = 0.0
    N = imgs.size(0)
    for i, pid in enumerate(pids):
        neg_imgs = imgs[pids != pid]
        neg_caps = imgs[pids != pid]
        pos_imgs = imgs[i:i+1].expand_as(neg_imgs)
        pos_caps = caps[i:i+1].expand_as(neg_caps)
        cap_tri_loss = triplet_cos_loss_inner(pos_imgs, pos_caps, neg_caps)
        img_tri_loss = triplet_cos_loss_inner(pos_caps, pos_imgs, neg_imgs)
        loss = loss + cap_tri_loss + img_tri_loss
    return loss / N

def triplet_cos_loss_attention(fulls, parts, pids):
    loss = 0.0
    N = fulls.size(0)
    
    for i, pid in enumerate(pids):
        curr_part = parts[i:i+1]
        curr_full = fulls[i:i+1]
        pos_part = RGA_attend_one_to_many_batch(curr_full, curr_part, 'cosine')
        
        neg_fulls = fulls[pids != pid]
        M, K = neg_fulls.size()
        neg_fulls = neg_fulls[:,None,:].expand(M, parts.size(1), K)
        neg_parts = RGA_attend_one_to_many_batch(neg_fulls, curr_part.expand_as(neg_fulls), 'cosine')
        
        loss = loss + triplet_cos_loss_inner(curr_full.expand_as(neg_parts), 
                                             pos_part.expand_as(neg_parts), 
                                             neg_parts)
    return loss / N
        
def regional_alignment_text(fulls, parts, p2fs, dist_fn_opt):
    start_index = 0
    aligned = []
    for i, jump in enumerate(p2fs):
        curr_parts = parts[start_index:start_index + jump]
        start_index += jump
        curr_full = fulls[i:i+1]
        aligned.append(RGA_attend_one_to_many(curr_full, curr_parts, dist_fn_opt))
    return torch.cat(aligned)

def regional_alignment_image(fulls, parts, dist_fn_opt):
    return RGA_attend_one_to_many_batch(fulls, parts, dist_fn_opt)
  
    
class Manager:
    def __init__(self, args, logger):
        self.log = logger.log
        self.cfg = args
        self._init_models()
        self._init_criterion()
    
    def _init_criterion(self):
        if self.cfg.dist_fn_opt == "cosine":
            self.triplet_loss = triplet_cos_loss
        elif self.cfg.dist_fn_opt == "euclidean":
            self.triplet_loss = nn.TripletMarginLoss()
        self.cls_loss = nn.CrossEntropyLoss()
        self.log("[Trainer][init] criterion initialized.")

    def _init_models(self):
        # encoder
        self.model = Model(embed_size=self.cfg.embed_size, 
                          image_opt=self.cfg.img_backbone_opt, 
                          caption_opt=self.cfg.cap_backbone_opt,
                          cap_embed_type=self.cfg.cap_embed_type,
                          img_num_cut=self.cfg.img_num_cut,
                          regional_embed_size=self.cfg.regional_embed_size).cuda()
        # id classifer
        self.id_cls = nn.Linear(self.cfg.embed_size, self.cfg.num_ids).cuda()
        # RGA image mlp
        self.rga_img_mlp = MLP(self.cfg.regional_embed_size, self.cfg.embed_size).cuda()
        # RGA text mlp
        self.rga_cap_mlp = MLP(self.cfg.embed_size, self.cfg.embed_size).cuda()
        # np encoder
        self.np_encoder = CaptionBackbone(embed_size=self.cfg.embed_size, 
                                          caption_opt=self.cfg.cap_backbone_opt,
                                          cap_embed_type=self.cfg.cap_embed_type).cuda()
        # gpu
        self.all_models = {
            "model": self.model,
            "id_cls": self.id_cls, 
            "rga_img_mlp": self.rga_img_mlp,
            "rga_cap_mlp": self.rga_cap_mlp,
            'np_encoder': self.np_encoder,
        }
        print(self.cfg.num_gpus)
        if self.cfg.num_gpus > 1:
            print('here')
            for name in self.all_models.keys():
                print('data parallel')
                self.all_models[name] = nn.DataParallel(self.all_models[name])
        # load ckpt
        self.reset_ckpt()
        
        
        self.log("[Trainer][init] model initialized.")

    def reset_ckpt(self):
        self.start_epoch = 0
        self.acc_history = []
        self.best_acc = ({'top-1':0, 'top-1': 0, 'top-1': 0}, self.start_epoch)
        if self.cfg.load_ckpt_fn == "0":
            self.log("[Trainer][init] initialize fresh model.")
            return
        ckpt = torch.load(self.cfg.load_ckpt_fn)
        self.start_epoch = ckpt["epoch"] + 1
        self.acc_history = ckpt["acc_history"]
        for name, network in self.all_models.items():
            if name in ckpt:
                network.load_state_dict(ckpt[name], strict=False)
                self.log("[Trainer][init] load pre-trained %s from %s." % (name, self.cfg.load_ckpt_fn))
              
    def save_ckpt(self, epoch, acc, fn):
        # update acc history 
        self.acc_history.append((acc, epoch))
        if acc['top-1'] > self.best_acc[0]['top-1']:
            self.best_acc = (acc, epoch)
        # ckpt 
        ckpt = {
            "epoch": epoch,
            "acc_history": self.acc_history,
            "best_acc": self.best_acc,
            }
        for name, network in self.all_models.items():
            ckpt[name] = network.module.state_dict() if isinstance(network, nn.DataParallel) else network.state_dict()

        path = os.path.join(self.cfg.model_path, fn)
        torch.save(ckpt, path)
            
    def todevice(self, batch):
        ret = []
        for arg in batch:
            if isinstance(arg, torch.Tensor):
                arg = arg.cuda()
            ret.append(arg)
        return tuple(ret)
    
    def melt_img_layer(self, num_layer_to_melt=1):
        if isinstance(self.model, nn.DataParallel):
            self.model.module.img_backbone.melt_layer(8 - num_layer_to_melt)
        else:
            self.model.img_backbone.melt_layer(8 - num_layer_to_melt)
     
    def train_epoch_global(self, train_data, optimizer, epoch, note="train"):
        self.model.train()
        cum_tri_loss, cum_id_loss = 0.0, 0.0
        for i, data in tqdm(enumerate(train_data), "%s, epoch%d" % (note, epoch)):
            # load data
            data = self.todevice(data)
            img, cap, pid = data
            
            # encode
            #img, pos_img, neg_img = self.model(img), self.model(pos_img), self.model(neg_img)
            #cap, pos_cap, neg_cap = self.model(cap), self.model(pos_cap), self.model(neg_cap)
            img, cap = self.model(img), self.model(cap)


            # loss
            tri_loss = triplet_cos_loss(img, cap, pid)
            id_loss = self.cls_loss(self.id_cls(img), pid) + self.cls_loss(self.id_cls(cap), pid)
            loss = tri_loss + id_loss

            # backpropagation
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            # log
            cum_tri_loss += tri_loss.item()
            cum_id_loss += id_loss.item()
            if (i+1) % self.cfg.print_freq == 0:
                out_string = "[ep-%d, bs-%d] " % (epoch, i)
                out_string += "[tri-loss] %.6f, " % (cum_tri_loss / self.cfg.print_freq)
                out_string += "[id-loss] %.6f, " % (cum_id_loss / self.cfg.print_freq)
                self.log(out_string)
                cum_tri_loss, cum_id_loss = 0.0, 0.0
                
    def train_epoch_regional(self, train_data, optimizer, epoch, note="train"):
        self.model.train(); self.rga_img_mlp.train(); self.rga_cap_mlp.train(); self.np_encoder.train()

        cum_tri_loss, cum_tri_image_regional_loss, cum_tri_text_regional_loss, cum_id_loss = 0.0, 0.0, 0.0, 0.0
        for i, data in tqdm(enumerate(train_data), "%s, epoch%d" % (note,epoch)):
            # load data
            data = self.todevice(data)
            (img, cap, nps, n2c, pid) = data
            # import pdb;pdb.set_trace()


            img, img_part = self.model(img)
            cap = self.model(cap)
            
            N, M, T = nps.size()
            #nps = self.rga_cap_mlp(self.np_encoder(nps))
            nps = self.rga_cap_mlp(self.model(nps.view(-1, T))).view(N, M, -1)
            
            # part
            img_part = self.rga_img_mlp(img_part)

            img_part = RGA_attend_one_to_many_batch(cap, img_part, self.cfg.dist_fn_opt)
            #cap_part = regional_alignment_text(img, nps, n2c, self.cfg.dist_fn_opt)
            cap_part = RGA_attend_one_to_many_batch(img, nps, self.cfg.dist_fn_opt)

            # loss
            tri_loss =  self.triplet_loss(img, cap, pid) 
            tri_text_regional_loss =  self.triplet_loss(img, cap_part, pid) 
            tri_image_regional_loss = self.triplet_loss(cap, img_part, pid) 
            id_loss = self.cls_loss(self.id_cls(img), pid) +  self.cls_loss(self.id_cls(cap), pid)
            id_loss = id_loss + self.cls_loss(self.id_cls(img_part), pid) +  self.cls_loss(self.id_cls(cap_part), pid)


            loss = tri_loss + tri_image_regional_loss  + tri_text_regional_loss + id_loss

            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            # log
            cum_tri_loss += tri_loss.item()
            cum_tri_image_regional_loss += tri_image_regional_loss.item()
            cum_tri_text_regional_loss += tri_text_regional_loss.item()
            cum_id_loss += id_loss.item()
            
            if (i+1) % self.cfg.print_freq == 0:
                out_string = "[ep-%d, bs-%d] " % (epoch, i)
                out_string += "[id-loss] %.6f, " % (cum_id_loss / self.cfg.print_freq)
                out_string += "[tri-loss] %.6f, " % (cum_tri_loss / self.cfg.print_freq)
                out_string += "[img_rga] %.6f, " %  (cum_tri_image_regional_loss / self.cfg.print_freq)
                out_string += "[cap_rga] %.6f " % (cum_tri_text_regional_loss / self.cfg.print_freq)
                self.log(out_string)
                cum_tri_loss, cum_tri_image_regional_loss, cum_tri_text_regional_loss, cum_id_loss = 0.0, 0.0, 0.0, 0.0       
    def train_epoch_id(self, train_data, optimizer, epoch, note="train"):
        self.model.train()
        self.id_cls.train()
        cum_loss = 0.0
        for i, data in tqdm(enumerate(train_data), "%s, epoch%d" % (note,epoch)):
            # load data
            data = self.todevice(data)
            (img, cap, pid) = data
            img = self.model(img)
            cap = self.model(cap)

            # loss
            loss = 0.0
            loss = loss + self.cls_loss(self.id_cls(img), pid) +  self.cls_loss(self.id_cls(cap), pid)
            cum_loss += loss.item()
            
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            # log
            if (i+1) % self.cfg.print_freq == 0:
                self.log("ep-%d, bs-%d, [id-loss] %.6f" % (epoch, i, cum_loss / self.cfg.print_freq))
                cum_loss = 0.0

In [ ]:
cfg.num_ids = len(train_loader.dataset.person2label.values())
manager = Manager(cfg, logger)

In [ ]:
if False:
    if cfg.np:
        acc = cos_evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
    else:
        acc = cos_evaluator.evaluate(manager.model)
    logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))


## Stage 1: ID Loss only

In [ ]:

if False:
    logger.log("======== [Stage 1] ============")
    manager.melt_img_layer(num_layer_to_melt=1)
    param_to_optimize = build_graph_optimizer([manager.model, manager.id_cls])
    optimizer = optim.Adam(param_to_optimize, lr=1e-3)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10)
    
    for epoch in range(0):
        manager.train_epoch_id(train_loader, optimizer, epoch, "train-stage-1")
        acc = evaluator.evaluate(manager.model)
        logger.log('[euclidean][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
        acc = cos_evaluator.evaluate(manager.model)
        logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (acc['top-1'], acc['top-5'], acc['top-10']))
        scheduler.step()
        manager.save_ckpt(epoch, acc, 'stage_1_id_last.pt')
    manager.save_ckpt(epoch, acc, 'id_initialized.pt')


## Stage 2: Matching + ID Loss

In [ ]:
from tqdm import tqdm_notebook as tqdm
manager.melt_img_layer(num_layer_to_melt=2)
param_to_optimize = build_graph_optimizer([manager.model, manager.id_cls])
optimizer = optim.Adam(param_to_optimize, lr=2e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10)
train_epoch = manager.train_epoch_regional if cfg.np else manager.train_epoch_global
for epoch in range(15):
    train_epoch(train_loader, optimizer, epoch, "train-stage-2")
    if cfg.np:
        cos_acc = cos_evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
    else:
        cos_acc = cos_evaluator.evaluate(manager.model)
    logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (cos_acc['top-1'], cos_acc['top-5'], cos_acc['top-10']))
    scheduler.step()

In [ ]:
manager.save_ckpt(epoch, acc, "match_stage2_trained.pt")

In [ ]:
if cfg.np:
    cos_acc = cos_evaluator.evaluate(manager.model, manager.rga_img_mlp, manager.rga_cap_mlp)
else:
    cos_acc = cos_evaluator.evaluate(manager.model)
logger.log('[cosine   ][global] R@1: %.4f | R@5: %.4f | R@10: %.4f' % (cos_acc['top-1'], cos_acc['top-5'], cos_acc['top-10']))
